In [ ]:
%%capture
%pip install datasets
%pip install transformers
%pip install scikit-learn
%pip install torch
#install if necessary, i'm using Python 3.12.7

UsageError: unrecognized arguments: out this line if there are any errors and diagnose


In [2]:
import pandas as pd
import json


def json_to_df(json_file):
    arrayForDF = []
    for current_subject in json_file['data']:
        subject = current_subject['title']
        for current_context in current_subject['paragraphs']:
            context = current_context['context']
            for current_question in current_context['qas']:
                question = current_question['question']
                if (len(question) > 2):
                    for answer in current_question['answers']:
                        answer_text = answer['text']
                        answer_start = answer['answer_start']

                        record = {
                                "answer_text": answer_text,
                                "answer_start": answer_start,
                                "question": question,
                                "context": context,
                                "subject": subject
                            }
                        arrayForDF.append(record)
    df = pd.DataFrame(arrayForDF)
    return df


json_file = open("dev-v1.1.json",)
data = json.load(json_file)
squad_df = json_to_df(data)
print(squad_df.head())


         answer_text  answer_start  \
0     Denver Broncos           177   
1     Denver Broncos           177   
2     Denver Broncos           177   
3  Carolina Panthers           249   
4  Carolina Panthers           249   

                                            question  \
0  Which NFL team represented the AFC at Super Bo...   
1  Which NFL team represented the AFC at Super Bo...   
2  Which NFL team represented the AFC at Super Bo...   
3  Which NFL team represented the NFC at Super Bo...   
4  Which NFL team represented the NFC at Super Bo...   

                                             context        subject  
0  Super Bowl 50 was an American football game to...  Super_Bowl_50  
1  Super Bowl 50 was an American football game to...  Super_Bowl_50  
2  Super Bowl 50 was an American football game to...  Super_Bowl_50  
3  Super Bowl 50 was an American football game to...  Super_Bowl_50  
4  Super Bowl 50 was an American football game to...  Super_Bowl_50  


In [3]:
from transformers import pipeline
import torch

distilBert=pipeline("question-answering",model="distilbert-base-uncased-distilled-squad")

c:\Users\nadun\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# you can skip the evaluation processing below and load squad_df from the next code block:
# model predictions took me 44 minutes and 50.3 seconds
# squad_df['model_prediction'] = squad_df.apply(
#     lambda row: distilBert(question=row['question'], context=row['context']),
#     axis=1
# )
# save above model predictions alongside SQuAD data 1.1 to csv.
# squad_df.to_csv('squad_df.csv',columns=["answer_text","question","context","model_prediction"], index=False)

In [6]:
# you can load squad_df straight from a file with model predictions
squad_df=pd.read_csv("squad_df.csv")
squad_df.head()

,answer_text,question,context,model_prediction
0,Denver Broncos,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,"{'score': 0.9883760213851929, 'start': 177, 'e..."
1,Denver Broncos,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,"{'score': 0.9883760213851929, 'start': 177, 'e..."
2,Denver Broncos,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,"{'score': 0.9883760213851929, 'start': 177, 'e..."
3,Carolina Panthers,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,"{'score': 0.9976987838745117, 'start': 249, 'e..."
4,Carolina Panthers,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,"{'score': 0.9976987838745117, 'start': 249, 'e..."


In [8]:
#convert model_prediction column to dictionary
import ast
squad_df['model_prediction'] = squad_df['model_prediction'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [10]:
#add columns
squad_df['answer'] = squad_df['model_prediction'].apply(lambda x: x.get('answer', None))
squad_df['score'] = squad_df['model_prediction'].apply(lambda x: x.get('score', None))
squad_df['exact_match'] = (squad_df['answer_text'] == squad_df['answer']).astype(int)
squad_df.head()

,answer_text,question,context,model_prediction,answer,score,exact_match
0,Denver Broncos,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,"{'score': 0.9883760213851929, 'start': 177, 'e...",Denver Broncos,0.988376,1
1,Denver Broncos,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,"{'score': 0.9883760213851929, 'start': 177, 'e...",Denver Broncos,0.988376,1
2,Denver Broncos,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,"{'score': 0.9883760213851929, 'start': 177, 'e...",Denver Broncos,0.988376,1
3,Carolina Panthers,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,"{'score': 0.9976987838745117, 'start': 249, 'e...",Carolina Panthers,0.997699,1
4,Carolina Panthers,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,"{'score': 0.9976987838745117, 'start': 249, 'e...",Carolina Panthers,0.997699,1


In [11]:
# calculate Exact Match (EM score)
average_score = squad_df[squad_df['exact_match'] == 1]['score'].mean()
print("Average Score of Exact Matches is:",average_score*100)

Average Score of Exact Matches is: 77.56786669267825


In [12]:
# calculate F1 score
from sklearn.metrics import f1_score

def calculate_f1(row):
    true_answer = str(row["answer_text"]).split()
    predicted_answer = str(row["answer"]).split()

    # Calculate precision and recall
    common = set(true_answer) & set(predicted_answer)
    precision = len(common) / len(predicted_answer) if predicted_answer else 0
    recall = len(common) / len(true_answer) if true_answer else 0

    # Calculate F1 score
    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * (precision * recall) / (precision + recall)
    
    return f1

# Apply the function to each row and store the F1 scores in a new column
squad_df['f1_score'] = squad_df.apply(calculate_f1, axis=1)
print("F1 Score:",squad_df["f1_score"].mean()*100)


F1 Score: 75.23371215677648
